In [2]:
from sklearn_pandas import DataFrameMapper

# other imports
from IPython.display import HTML
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
my_favorite_shows = {
    'name': ['sense_8',
             'handmaidens_tale',
             'the_good_place',
             'big_little_lies',
             'jane_the_virgin',
             'game_of_thrones',
             'mad_men',
             'the_crown',
             'narcos',
             'house_of_cards',
             'girls',
             'breaking_bad',
             'bad_blood',
             'fauda',
             'jessica_jones'],
    'cost': [None,
             140000000,
             80000000,
             170000000,
             205000000,
             600000000,
             300000000,
             None,
             400000000,
             500000000,
             112000000,
             380000000,
             10000000,
             75000000,
             None],
    'seasons': [2, None, 3, 1, 5, 9, 7, 2, 3, 5, 6, 5, 2, None, 2],
    'popularity': [5.8, 6, 5.7, 7.3, 6.5, 9.8, 8.4,
                   7.6, 8, 9.3, 7, 8.9, 2.3, 5.2, 4.7],
    'genre': ['science_fiction',
              'speculative_fiction',
              'comedy',
              'drama',
              'comedy',
              'fantasy',
              'period_drama',
              'period_drama',
              'period_drama',
               None,
              'comedy',
              'crime',
              'crime',
              'crime',
              'science_fiction'],
    'on_netflix': ['yes', 'no', 'yes', 'no', 'yes', 'no', 'yes',
                   'yes', None, 'yes', 'no', 'yes', None, 'yes', 'yes']
}

my_favorite_shows = pd.DataFrame(my_favorite_shows)
HTML(my_favorite_shows.head(10).to_html(classes="table table-stripped table-hover table-dark"))

,name,cost,seasons,popularity,genre,on_netflix
0,sense_8,NaN,2.0,5.8,science_fiction,yes
1,handmaidens_tale,140000000.0,NaN,6.0,speculative_fiction,no
2,the_good_place,80000000.0,3.0,5.7,comedy,yes
3,big_little_lies,170000000.0,1.0,7.3,drama,no
4,jane_the_virgin,205000000.0,5.0,6.5,comedy,yes
5,game_of_thrones,600000000.0,9.0,9.8,fantasy,no
6,mad_men,300000000.0,7.0,8.4,period_drama,yes
7,the_crown,NaN,2.0,7.6,period_drama,yes
8,narcos,400000000.0,3.0,8.0,period_drama,None
9,house_of_cards,500000000.0,5.0,9.3,None,yes


In [ ]:
from sklearn.model_selection import train_test_split
X = my_favorite_shows.drop(columns=['name', 'popularity'], axis=1)
y = my_favorite_shows['popularity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
HTML(X_train.head(10).to_html(classes="table table-stripped table-hover table-dark"))

In [ ]:
median_budget = X_train['cost'].quantile(0.5)
median_season = X_train['seasons'].quantile(0.5)
X_train['cost'] = X_train['cost'].fillna(median_budget)
X_train['genre'] = X_train['genre'].fillna('unknown')
X_train['seasons'] = X_train['seasons'].fillna(median_season)
X_train['on_netflix'] = X_train['on_netflix'].fillna('unknown')

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
binarized = pd.DataFrame(lb.fit_transform(X_train['genre']), columns=list(lb.classes_))
HTML(binarized.head(5).to_html(classes="table table-stripped table-hover table-dark"))

In [ ]:
X_train.drop(columns=['genre'], axis=1, inplace=True)
Z_train = pd.merge(X_train, binarized, how='left', on = X_train.index)
HTML(Z_train.head(5).to_html(classes="table table-stripped table-hover table-dark"))

In [ ]:
Z_train.set_index('key_0', inplace=True)
HTML(Z_train.head(5).to_html(classes="table table-stripped table-hover table-dark"))

In [ ]:
Z_train['on_netflix'] = Z_train['on_netflix'].replace({'no': 0, 'yes': 2, 'unknown': 1})

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(Z_train, y_train)
train_score = model.score(Z_train, y_train)
print(f'Train score: {train_score}')

In [ ]:
X_test['cost'] = X_test['cost'].fillna(median_budget)
X_test['genre'] = X_test['genre'].fillna('unknown')
X_test['seasons'] = X_test['seasons'].fillna(median_season)
X_test['on_netflix'] = X_test['on_netflix'].fillna('unknown')
binarized = pd.DataFrame(lb.transform(X_test['genre']), columns=list(lb.classes_))
X_test.drop(columns=['genre'], axis=1, inplace=True)
Z_test = pd.merge(X_test, binarized, how='left', on = X_test.index)
Z_test['on_netflix'] = Z_test['on_netflix'].replace({'no': 0, 'yes': 2, 'unknown': 1})
Z_test.set_index('key_0', inplace=True)

In [ ]:
test_score = model.score(Z_test, y_test)
print(f'Test score: {test_score}')

In [ ]:
print(f'Predicted scores: {list(model.predict(Z_test))}')
print(f'Actual scores: {list(y_test)}')

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

X = my_favorite_shows.drop(columns=['name', 'popularity'], axis=1)
y = my_favorite_shows['popularity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

HTML(X_train.head(5).to_html(classes="table table-stripped table-hover table-dark"))

,cost,seasons,genre,on_netflix
5,600000000.0,9.0,fantasy,no
3,170000000.0,1.0,drama,no
9,500000000.0,5.0,None,yes
8,400000000.0,3.0,period_drama,None
1,140000000.0,NaN,speculative_fiction,no


In [9]:
#X_train['genre'].values = X_train['genre'].shape(1,-1)
print(X_train)

           cost  seasons                genre on_netflix
5   600000000.0      9.0              fantasy         no
3   170000000.0      1.0                drama         no
9   500000000.0      5.0                 None        yes
8   400000000.0      3.0         period_drama       None
1   140000000.0      NaN  speculative_fiction         no
4   205000000.0      5.0               comedy        yes
14          NaN      2.0      science_fiction        yes
6   300000000.0      7.0         period_drama        yes
13   75000000.0      NaN                crime        yes
10  112000000.0      6.0               comedy         no
0           NaN      2.0      science_fiction        yes
2    80000000.0      3.0               comedy        yes


In [5]:
mapper = DataFrameMapper([
    (['cost'], SimpleImputer(strategy='median')),
    (['seasons'], SimpleImputer(strategy='median')),
    ('genre', [SimpleImputer(strategy='constant', fill_value='unknown'),LabelEncoder()]),
               
    #('on_netflix', [SimpleImputer(strategy='constant', fill_value='unknown'),
    #               LabelEncoder()])
], df_out=True)
Z_train = mapper.fit_transform(X_train)
HTML(Z_train.head(5).to_html(classes="table table-stripped table-hover table-dark"))

ValueError: genre: Expected 2D array, got 1D array instead:
array=['fantasy' 'drama' None 'period_drama' 'speculative_fiction' 'comedy'
 'science_fiction' 'period_drama' 'crime' 'comedy' 'science_fiction'
 'comedy'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.